# Assigning Surname IDs

#### Imports

In [3]:
import json
import pandas as pd
import numpy as np
import phonetics as ph
import re
from tqdm.notebook import tqdm
tqdm.pandas()
import os
import unidecode
import platform
import torch
import torch.nn as nn
import torch.nn.functional as F
from rapidfuzz import fuzz
import time
if platform.node() == 'Nick_Laptop':
    drive = 'C'
elif platform.node() == 'MSI':
    drive = 'D'
else:
    drive = 'uhhhhhh'
    print('Uhhhhhhhhhhhhh')
os.chdir(f'{drive}:/PhD/DissolutionProgramming/LND---Land-Paper')
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#%% Globals
PROCESSED = 'Data/Processed'
RAW = 'Data/Raw'
MODELS = f'Code/ml_models/'
SURNAMES = f'{PROCESSED}/surname_info'

#### Regexes

In [4]:

alt_curly_brackets = re.compile(r'([A-Za-z]+)\s+\{[A-Za-z]+\}')
single_curly_brackets = re.compile(r'\{([A-Za-z]+)\}')
alias_pattern = re.compile(r'([A-Za-z]+) ali?a?s ([A-Za-z]+)')
or_pattern = re.compile(r'([A-Za-z]+) or ([A-Za-z]+)')
double_name = re.compile(r'([A-Za-z]+)\s+([A-Za-z]+)')
and_sons = re.compile(r'([A-Za-z]+) & sons+', re.IGNORECASE)
and_co = re.compile(r'([A-Za-z]+) & co', re.IGNORECASE)
saint_pattern = re.compile(r'Sa?i?n?t\s+([A-Za-z]+)')


#### Defining and Loading the Model

In [5]:
#%% Defining model
def encode_surname(surname, max_len=24):
    CHARSET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0'
    CHARSET_DICT = {char: i + 1 for i, char in enumerate(CHARSET)}
    PAD = 0
    surname = surname.upper()
    surname = ''.join([char for char in surname if char in CHARSET])
    metaphone = ph.metaphone(surname)

    encoded = [CHARSET_DICT[char] for char in surname]
    if len(encoded) < max_len:
        encoded += [PAD] * (max_len - len(encoded))
    encoded = torch.tensor(encoded).long()

    encoded_metaphone = [CHARSET_DICT[char] for char in metaphone]
    if len(encoded_metaphone) < max_len:
        encoded_metaphone += [PAD] * (max_len - len(encoded_metaphone))
    encoded_metaphone = torch.tensor(encoded_metaphone).long()

    return encoded, encoded_metaphone

class CrossEncoder(nn.Module):
    def __init__(self, embed_dim=128, hidden_dim=64, fc_dim=32):
        super(CrossEncoder, self).__init__()

        # Embedding layer (shared between names and metaphones)
        self.name_embedding = nn.Embedding(28, embed_dim)  # Assuming 27 letters + 1 padding
        self.metaphone_embedding = nn.Embedding(28, embed_dim)
        # BiLSTM for sequence encoding
        self.name_lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.metaphone_lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)

        # Fully connected layers for classification
        self.fc1 = nn.Linear(4*2 * hidden_dim, fc_dim)  # Combining all four encodings
        self.fc2 = nn.Linear(fc_dim, 1)

    def name_encode(self, x):
        name_embedded = self.name_embedding(x)
        _, (name_hidden, _) = self.name_lstm(name_embedded)
        name_hidden = torch.cat((name_hidden[0], name_hidden[1]), dim=1)  # Concatenate forward & backward LSTM outputs
        return name_hidden
    def metaphone_encode(self, x):
        metaphone_embedded = self.metaphone_embedding(x)
        _, (metaphone_hidden, _) = self.metaphone_lstm(metaphone_embedded)
        metaphone_hidden = torch.cat((metaphone_hidden[0], metaphone_hidden[1]), dim=1)  # Concatenate forward & backward LSTM outputs
        return metaphone_hidden

    def forward(self, name1, metaphone1, name2, metaphone2):
        # Encode each input separately
        name1_encoded = self.name_encode(name1)
        metaphone1_encoded = self.metaphone_encode(metaphone1)
        name2_encoded = self.name_encode(name2)
        metaphone2_encoded = self.metaphone_encode(metaphone2)
        # Concatenate all representations
        combined = torch.cat((name1_encoded, metaphone1_encoded, name2_encoded, metaphone2_encoded), dim=1)

        # Fully connected layers
        fc1_out = self.fc1(combined)
        fc1_relud = F.relu(fc1_out)
        output = torch.sigmoid(self.fc2(fc1_relud))  # Binary classification

        return output

# Load the model
model = CrossEncoder()
model.load_state_dict(torch.load(f'{MODELS}/name_matcher/cross_encoder_1.pth'))
model.eval()
model.to(device)


CrossEncoder(
  (name_embedding): Embedding(28, 128)
  (metaphone_embedding): Embedding(28, 128)
  (name_lstm): LSTM(128, 64, batch_first=True, bidirectional=True)
  (metaphone_lstm): LSTM(128, 64, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=512, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)

#### Surname Lists, Building Surname ID df

In [6]:
with open(f'{SURNAMES}/combined_surnames.json') as f:
    combined_surname_lists = json.load(f)

with open(f'{SURNAMES}/non_combined_surnames.json') as f:
    non_combined_surname_lists = json.load(f)

all_unique_surnames = set()
for surname_list in non_combined_surname_lists:
    all_unique_surnames.update(surname_list)

#1. One ID per unique surname spelling
unique_id_dict = zip(all_unique_surnames, range(1, len(all_unique_surnames)+1))
unique_id_dict = dict(unique_id_dict)

#2. One ID per surname group (not combined)
group_id_dict = {}
group_id_dict_2 = {}
for i, surname_list in enumerate(non_combined_surname_lists):
    for surname in surname_list:
        if surname in group_id_dict:
            if len(surname_list) == 1:
                continue
            other_list = [x for x in group_id_dict.keys() if group_id_dict[x] == group_id_dict[surname]]
            if len(other_list) != 1: 
                group_id_dict_2[surname] = i+1
                continue
        group_id_dict[surname] = i+1

#3. One ID per combined surname group
combined_id_dict = {}
for i, surname_list in enumerate(combined_surname_lists):
    for surname in surname_list:
        combined_id_dict[surname] = i + 1

#4. One ID per metaphone code
metaphone_id_dict = {}
surnames_metaphone = [ph.metaphone(''.join(unidecode.unidecode(char) for char in x if char.isalpha()).title()) for x in all_unique_surnames]
surnames_metaphone = list(set(surnames_metaphone))
for i, surname in enumerate(surnames_metaphone):
    metaphone_id_dict[surname] = i + 1

#5. One ID to run them all, one ID to find them; one ID to bring them all and in the darkness bind them
# This will be based on the non-combined surname lists, but will use a round of metaphone and a round of ML before assigning a new ID
master_id_dict = combined_id_dict.copy()


print(len(unique_id_dict), 'unique surnames')
print(len(group_id_dict), 'non-combined surname groups')
print(len(combined_id_dict), 'combined surname groups')
print(len(metaphone_id_dict), 'unique metaphone codes')
print(len(master_id_dict), 'master IDs')

33348 unique surnames
33348 non-combined surname groups
33407 combined surname groups
8123 unique metaphone codes
33407 master IDs


#### Pre-Compute Metaphone and Encoding for master_id_dict

In [7]:
master_id_data = {}
for surname, master_id in tqdm(master_id_dict.items()):
    metaphone = ph.metaphone(''.join(unidecode.unidecode(char) for char in surname if char.isalpha()).title())
    surname_tensor, metaphone_tensor = encode_surname(surname)
    master_id_data[surname] = {
        'surname': surname,
        'metaphone': metaphone,
        'master_id': master_id,
        'surname_tensor': surname_tensor,
        'metaphone_tensor': metaphone_tensor
    }


  0%|          | 0/33407 [00:00<?, ?it/s]

#### Functions for Cleaning Surnames, Applying IDs

In [8]:
def clean_surnames(df):
    # Replace number characters with letters and remove special characters
    replacements = {
        'NO_SURNAME': np.nan, '#NAME': np.nan,
        '1': 'l', '2': 'z', '3': 'e', '4': 'a', '5': 's', 
        '6': 'b', '7': 't', '8': 'b', '9': 'g', '0': 'o',
        'ĩ': 'i', 'ó': 'o'
    }

    # All special characters to remove
    chars_to_remove = [',', '.', '\'', '(', ')', '-', '/', '?', '­', ';', ':', 
                    ''', ''', '"', '"', '•', '!', '"', '&', '{', '}', '[', ']']

    # Apply replacements
    df['surname'] = df['surname'].replace(replacements)

    # Remove all special characters
    for char in chars_to_remove:
        df['surname'] = df['surname'].str.replace(char, '')

    # Apply regex replacements
    regex_patterns = [
        (alt_curly_brackets, r'\1'), 
        (single_curly_brackets, r'\1'),
        (alias_pattern, r'\1'),
        (or_pattern, r'\1'),
        (and_sons, r'\1'),
        (and_co, r'\1'),
        (saint_pattern, r'\1')
    ]

    for pattern, replacement in regex_patterns:
        df['surname'] = df['surname'].str.replace(pattern, replacement, regex=True)

    # Clean up whitespace and apply double_name pattern twice
    df['surname'] = df['surname'].str.strip()
    df['surname'] = df['surname'].str.replace(double_name, r'\1', regex=True)
    df['surname'] = df['surname'].str.strip()
    df['surname'] = df['surname'].str.replace(double_name, r'\1', regex=True)
    df['surname'] = df['surname'].str.strip()

    df = df[df['surname'] != '']
    df = df[df['surname'] != 'None Qualified']
    df = df[df['surname'] != 'NO_SURNAME']
    df = df[df['surname'] != 'No Entries']
    df = df.loc[df['surname'].notna()]
    df['surname'] = df['surname'].progress_apply(lambda x: ''.join(unidecode.unidecode(char) for char in x if char.isalpha()).title())

    return df


def assign_unique_id(df, unique_id_dict):
    df['unique_id'] = np.nan
    df['unique_id'] = df['surname'].map(unique_id_dict)
    for i, row in df.iterrows():
        if pd.isna(row['unique_id']):
            df.at[i, 'unique_id'] = len(unique_id_dict) + 1
            unique_id_dict[row['surname']] = df.at[i, 'unique_id']
    df['unique_id'] = df['unique_id'].astype(int)
    return df, unique_id_dict

def assign_group_id(df, group_id_dict, group_id_dict_2):
    df['group_id'] = np.nan
    for i, row in df.iterrows():
        surname = row['surname']
        if surname in group_id_dict_2:
            id_1 = group_id_dict[surname]
            id_2 = group_id_dict_2[surname]

            group_1 = [k for k, v in group_id_dict.items() if v == id_1]
            group_2 = [k for k, v in group_id_dict.items() if v == id_2]

            # Get mean metaphone edit distance from surname to each group
            surname_metaphone = ph.metaphone(surname)
            group_1_metaphone = [ph.metaphone(x) for x in group_1]
            group_2_metaphone = [ph.metaphone(x) for x in group_2]
            group_1_metaphone = list(set(group_1_metaphone))
            group_2_metaphone = list(set(group_2_metaphone))
            group_1_scores = [fuzz.ratio(surname_metaphone, x) for x in group_1_metaphone]
            group_2_scores = [fuzz.ratio(surname_metaphone, x) for x in group_2_metaphone]
            group_1_score = np.mean(group_1_scores)
            group_2_score = np.mean(group_2_scores)
            if group_2_score > group_1_score:
                df.at[i, 'group_id'] = id_2
            else:
                df.at[i, 'group_id'] = id_1
            continue
        if surname in group_id_dict:
            df.at[i, 'group_id'] = group_id_dict[surname]
        else:
            next_id = max(group_id_dict.values()) + 1
            df.at[i, 'group_id'] = next_id
            group_id_dict[surname] = next_id
    df['group_id'] = df['group_id'].astype(int)
    return df, group_id_dict

def assign_combined_id(df, combined_id_dict):
    df['combined_id'] = np.nan
    for i, row in df.iterrows():
        surname = row['surname']
        if surname in combined_id_dict:
            df.at[i, 'combined_id'] = combined_id_dict[surname]
        else:
            next_id = max(combined_id_dict.values()) + 1
            df.at[i, 'combined_id'] = next_id
            combined_id_dict[surname] = next_id
    df['combined_id'] = df['combined_id'].astype(int)
    return df, combined_id_dict

def assign_metaphone_id(df, metaphone_id_dict):
    df['metaphone_id'] = np.nan
    for i, row in df.iterrows():
        surname = row['surname']
        metaphone = ph.metaphone(surname)
        if metaphone in metaphone_id_dict:
            df.at[i, 'metaphone_id'] = metaphone_id_dict[metaphone]
        else:
            next_id = max(metaphone_id_dict.values()) + 1
            df.at[i, 'metaphone_id'] = next_id
            metaphone_id_dict[metaphone] = next_id
    df['metaphone_id'] = df['metaphone_id'].astype(int)
    return df, metaphone_id_dict

def assign_master_id(df, master_id_dict):
    df['master_id'] = np.nan
    # Insta-assigning IDs if they're in the dict
    df['master_id'] = df['surname'].map(master_id_dict)
    for i, row in tqdm(df.iterrows(), total=len(df)):
        surname = row['surname']
        if not pd.isna(row['master_id']):
            continue
        # If the surname is already in the dict, we can just assign it
        if surname in master_id_dict:
            df.at[i, 'master_id'] = master_id_dict[surname]
            continue
        # If the surname isn't already in the dict, we need to check for candidate matches
        ## First round: metaphone
        candidate_ids = []
        metaphone = ph.metaphone(surname)
        for key, data in master_id_data.items():
            fuzz_ratio = fuzz.ratio(metaphone, data['metaphone'])/100
            if len(metaphone) - (fuzz_ratio * len(metaphone)) <= 1:
                candidate_ids.append(data['master_id'])
        # If we have a single winner, we assign it
        candidate_ids = list(set(candidate_ids))
        if len(candidate_ids) == 1:
            df.at[i, 'master_id'] = candidate_ids[0]
            continue
        # If there's no similar-sounding candidates, we just assign a new ID and add its info to the dict
        if len(candidate_ids) == 0:
            next_id = max(master_id_dict.values()) + 1
            df.at[i, 'master_id'] = next_id
            master_id_dict[surname] = next_id
            # Add the surname to the master_id_data dict
            surname_tensor, metaphone_tensor = encode_surname(surname)
            master_id_data[surname] = {
                'surname': surname,
                'metaphone': metaphone,
                'master_id': next_id,
                'surname_tensor': surname_tensor,
                'metaphone_tensor': metaphone_tensor
            }
            continue
        mid_time = time.time()
        ## Second round: ML

        surname_tensor, metaphone_tensor = encode_surname(surname)
        surname_tensor = surname_tensor.to(device)
        metaphone_tensor = metaphone_tensor.to(device)
        # Assembling our candidate lists
        id_scores = []
        id_indices = {}
        index = 0
        candidate_surname_tensors = []
        candidate_metaphone_tensors = []
        print(len(candidate_ids))
        for candidate_id in candidate_ids:
            candidate_surnames = [key for key, data in master_id_data.items() if data['master_id'] == candidate_id]
            if len(candidate_ids) > 10000:
                # If there are too many candidates, we sample them to speed up the process
                if len(candidate_surnames) > 3:
                    candidate_surnames = np.random.choice(candidate_surnames, size=3, replace=False).tolist()
            
            for candidate_surname in candidate_surnames:
                candidate_surname_tensor = master_id_data[candidate_surname]['surname_tensor']
                candidate_surname_tensor = candidate_surname_tensor.to(device)
                candidate_metaphone_tensor = master_id_data[candidate_surname]['metaphone_tensor']
                candidate_metaphone_tensor = candidate_metaphone_tensor.to(device)
                candidate_surname_tensors.append(candidate_surname_tensor)
                candidate_metaphone_tensors.append(candidate_metaphone_tensor)

                index_list = [index]
                index_list.append(index)
                if candidate_id not in id_indices:
                    id_indices[candidate_id] = []
                id_indices[candidate_id].extend([index, index + 1])
                index += 1

        candidate_surname_tensors_stacked = torch.stack(candidate_surname_tensors)
        candidate_surname_tensors_stacked = candidate_surname_tensors_stacked.to(device)
        candidate_metaphone_tensors_stacked = torch.stack(candidate_metaphone_tensors)
        candidate_metaphone_tensors_stacked = candidate_metaphone_tensors_stacked.to(device)
        batch_size = len(candidate_surname_tensors_stacked)
        # Reshape the tensors to match the expected input shape
        surname_tensor_batched = surname_tensor.unsqueeze(0).expand(batch_size, -1)
        metaphone_tensor_batched = metaphone_tensor.unsqueeze(0).expand(batch_size, -1)
        surname_tensor_batched = surname_tensor_batched.to(device)
        metaphone_tensor_batched = metaphone_tensor_batched.to(device)
        # Forward pass
        with torch.no_grad():
            output = model(surname_tensor_batched, metaphone_tensor_batched, candidate_surname_tensors_stacked, candidate_metaphone_tensors_stacked)
        for k, v in id_indices.items():
            
            if len(v) == 0:
                continue
            id_scores.append(output[v[0]:v[-1]].max())
        del candidate_surname_tensors_stacked
        del candidate_metaphone_tensors_stacked
        id_scores = torch.stack(id_scores).cpu().numpy()
        if max(id_scores) < 0.99:
            next_id = max(master_id_dict.values()) + 1
            df.at[i, 'master_id'] = next_id
            master_id_dict[surname] = next_id
            # Add the surname to the master_id_data dict
            surname_tensor, metaphone_tensor = encode_surname(surname)
            master_id_data[surname] = {
                'surname': surname,
                'metaphone': metaphone,
                'master_id': next_id,
                'surname_tensor': surname_tensor,
                'metaphone_tensor': metaphone_tensor
            }
            continue
        best_id_index = np.argmax(id_scores)
        best_id = candidate_ids[best_id_index]
        df.at[i, 'master_id'] = best_id
        # Add the surname to the master_id_dict
        master_id_dict[surname] = best_id
        # Send candidate tensors back to CPU
        candidate_surname_tensors = [x.detach().cpu() for x in candidate_surname_tensors]
        candidate_metaphone_tensors = [x.detach().cpu() for x in candidate_metaphone_tensors]
        
        # Add the surname to the master_id_data dict
        surname_tensor, metaphone_tensor = encode_surname(surname)
        master_id_data[surname] = {
            'surname': surname,
            'metaphone': metaphone,
            'master_id': best_id,
            'surname_tensor': surname_tensor.cpu(),
            'metaphone_tensor': metaphone_tensor.cpu()
        }
        torch.cuda.empty_cache()
    df['master_id'] = df['master_id'].astype(int)
    return df, master_id_dict

            

#### Applying IDs to all name lists

In [9]:
master_subsidy_list = []

for doc_file in [
    f'{PROCESSED}/calendar_recipients.csv',
    f'{PROCESSED}/master_subsidy_data.csv',
    f'{PROCESSED}/tithe_landowners.csv',
    f'{RAW}/freeholders_list_1713_1780.csv',
    f'{RAW}/bank_returns_1845_1880.csv',
    f'{RAW}/bankrupts_list_1800_1820.csv',
    f'{RAW}/bankrupts_list_1820_1843.csv',
    f'{RAW}/indictable_offenses_1745_1782.csv',
    f'{RAW}/monumental_brasses.csv',
    f'{RAW}/victuallers_list_1651_1828.csv',
    f'{RAW}/workhouse_list_1861.csv',
    f'{PROCESSED}/ukda_pcc_wills.csv'
     ]:
    tdf = pd.read_csv(doc_file, encoding='utf-8')
    if 'subsidy' in doc_file:
        surname_col = 'gemini_surname'
    elif 'tithe' in doc_file:
        surname_col = 'owner_surname'
    else:
        surname_col = 'surname'
        tdf = tdf[tdf['surname'] != '[No entries]']
        tdf = tdf[tdf['surname'] != 'None Qualified']
    
    tdf = tdf.rename(columns={surname_col: 'surname'})
    

    tdf = clean_surnames(tdf)

    tdf['metaphone'] = tdf['surname'].progress_apply(ph.metaphone)
    
    tdf, unique_id_dict = assign_unique_id(tdf, unique_id_dict)
    print(f'Unique IDs assigned for {doc_file}')
    tdf, group_id_dict = assign_group_id(tdf, group_id_dict, group_id_dict_2)
    print(f'Group IDs assigned for {doc_file}')
    tdf, combined_id_dict = assign_combined_id(tdf, combined_id_dict)
    print(f'Combined IDs assigned for {doc_file}')
    tdf, metaphone_id_dict = assign_metaphone_id(tdf, metaphone_id_dict)
    print(f'Metaphone IDs assigned for {doc_file}')
    tdf, master_id_dict = assign_master_id(tdf, master_id_dict)
    print(f'Master IDs assigned for {doc_file}')
    doc_file = doc_file.replace(RAW, PROCESSED)
    doc_file = doc_file.replace('.csv', '_final.csv')
    tdf.to_csv(doc_file, index=False, encoding='utf-8')
    print('='*20)
# Save all dictionaries when done
with open(f'{SURNAMES}/unique_id_dict.json', 'w') as f:
    json.dump(unique_id_dict, f)
with open(f'{SURNAMES}/group_id_dict.json', 'w') as f:
    json.dump(group_id_dict, f)
with open(f'{SURNAMES}/combined_id_dict.json', 'w') as f:
    json.dump(combined_id_dict, f)
with open(f'{SURNAMES}/metaphone_id_dict.json', 'w') as f:
    json.dump(metaphone_id_dict, f)
with open(f'{SURNAMES}/master_id_dict.json', 'w') as f:
    json.dump(master_id_dict, f)

  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/176 [00:00<?, ?it/s]

Unique IDs assigned for Data/Processed/calendar_recipients.csv
Group IDs assigned for Data/Processed/calendar_recipients.csv
Combined IDs assigned for Data/Processed/calendar_recipients.csv
Metaphone IDs assigned for Data/Processed/calendar_recipients.csv


c:\PhD\DissolutionProgramming\LND---Land-Paper\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\PhD\DissolutionProgramming\LND---Land-Paper\.venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/176 [00:00<?, ?it/s]

...

Unique IDs assigned for Data/Processed/ukda_pcc_wills.csv


c:\PhD\DissolutionProgramming\LND---Land-Paper\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\PhD\DissolutionProgramming\LND---Land-Paper\.venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Group IDs assigned for Data/Processed/ukda_pcc_wills.csv
Combined IDs assigned for Data/Processed/ukda_pcc_wills.csv
Metaphone IDs assigned for Data/Processed/ukda_pcc_wills.csv


  0%|          | 0/27034 [00:00<?, ?it/s]

219
172
1641
53
2503
1293
246
5
362
45
291
7
418
836
2
1686
8
129
473
66
77
309
2340
366
541
51
153
815
240
1224
544
584
315
1530
378
2126
374
2126
52
75
532
736
153
8
43
382
371
892
10
6
104
112
515
122
149
1108
411
10
115
361
199
166
169
2341
45
1333
49
36
9
70
318
398
183
520
218
841
710
159
6
227
6
894
246
695
384
74
127
280
691
494
167
5
22
46
66
323
376
6
14
46
690
75
40
28
1010
1010
57
812
665
39
46
51
40
1913
487
41
105
53
199
139
984
6
440
1175
546
304
32
290
125
87
29
17
58
282
569
186
388
154
184
1437
35
777
233
436
585
23
940
845
361
1063
26
1763
1018
1175
216
87
259
1437
69
1437
373
51
436
572
1839
254
1333
445
79
372
5
1170
11
27
585
331
422
692
21
372
691
11
382
275
49
198
48
264
151
256
263
66
1010
1618
87
146
515
69
755
234
158
1116
31
16
301
246
12072
152
210
721
515
736
62
76
449
841
710
154
1013
319
808
670
127
961
176
169
736
66
623
225
1150
1531
66
1686
25
449
487
206
550
10
378
378
6
238
32
422
367
66
350
40
199
367
420
385
53
295
350
541
1170
2505
749
120
664
14

In [10]:
import json
import os
import platform

if platform.node() == 'Nick_Laptop':
    drive = 'C'
elif platform.node() == 'MSI':
    drive = 'D' 
else:
    drive = 'uhhhhhh'
    print('Uhhhhhhhhhhhhh')
os.chdir(f'{drive}:/PhD/DissolutionProgramming/LND---Land-Paper')
PROCESSED = 'Data/Processed'
SURNAMES = f'{PROCESSED}/surname_info'
with open(f'{SURNAMES}/unique_id_dict.json', 'r') as f:
    unique_id_dict = json.load(f)
with open(f'{SURNAMES}/group_id_dict.json', 'r') as f:
    group_id_dict = json.load(f)
with open(f'{SURNAMES}/combined_id_dict.json', 'r') as f:
    combined_id_dict = json.load(f)
with open(f'{SURNAMES}/metaphone_id_dict.json', 'r') as f:
    metaphone_id_dict = json.load(f)
with open(f'{SURNAMES}/master_id_dict.json', 'r') as f:
    master_id_dict = json.load(f)

# Find max id for each dict and print
print('Max unique ID:', max(unique_id_dict.values()))
print('Max group ID:', max(group_id_dict.values()))
print('Max combined ID:', max(combined_id_dict.values()))
print('Max metaphone ID:', max(metaphone_id_dict.values()))
print('Max master ID:', max(master_id_dict.values()))

Max unique ID: 56874.0
Max group ID: 39580
Max combined ID: 34883
Max metaphone ID: 10827
Max master ID: 12145
